The goal of this notebook is to retrieve the times of the 88D and the kas that are the same so I can determine the proper distance between the two (lat/lon).

In [2]:
# import necessary stuff
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import coords
import xarray as xr
import pyart
from datetime import datetime as dt
from datetime import timedelta
from radarcalc import *
import matplotlib.pyplot as plt
import metpy.calc as mpcalc
import metpy
import metpy.plots
from metpy.units import units
import cartopy.crs as ccrs
import cartopy
import gc

/var/folders/4f/9s_4__q16tx5t8vmww4mv03h0000gn/T/ipykernel_79754/4185104306.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd



## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [103]:
# gps data for first ka
ka1gps = pd.read_csv('/Users/juliabman/Desktop/research2024/GPS_Ka1_20220523.txt')
# scan data for first ka
ka1scan = pd.read_csv('/Users/juliabman/Desktop/research2024/20220523_Ka1_scan_log.txt', dtype=str)

# dealiased vad data created from VADS.ipynb for ka 1, but not bearing corrected
#vad_fileska1 = sorted(glob.glob('/Users/juliabman/Desktop/research2024/product_raw_ka1/*.nc'))
vad_fileska1 = sorted(glob.glob('/Users/juliabman/Desktop/dealiased_data/Ka1*.nc'))

# sanity checks to make sure they're loading in:
#print(vad_fileska1)
#print(ka1gps)

In [83]:
ka1gps['hhmmss[UTC]']

0        183124
1        183125
2        183126
3        183127
4        183128
          ...  
31901     32353
31902     32354
31903     32355
31904     32356
31905     32357
Name: hhmmss[UTC], Length: 31906, dtype: int64

In [84]:
file = vad_fileska1[10]

In [85]:
file

'/Users/juliabman/Desktop/research2024/product_raw_ka1/Ka1220523222620.nc'

In [86]:
radar = pyart.io.read_cfradial(file)
radar.latitude['data']

masked_array(data=[43.53127012],
             mask=False,
       fill_value=1e+20)

In [89]:
vad_ka1_times = []
for time_grab in range(len(vad_fileska1)):
    file = vad_fileska1[time_grab]
    time_yoink = file[-15:-3]
    time_yoink_datetime = dt.strptime(time_yoink, '%y%m%d%H%M%S')
    vad_ka1_times.append(time_yoink_datetime)

In [90]:
vad_ka1_times

[datetime.datetime(2022, 5, 23, 20, 56, 32),
 datetime.datetime(2022, 5, 23, 20, 57, 1),
 datetime.datetime(2022, 5, 23, 20, 57, 30),
 datetime.datetime(2022, 5, 23, 20, 57, 59),
 datetime.datetime(2022, 5, 23, 20, 58, 28),
 datetime.datetime(2022, 5, 23, 20, 58, 57),
 datetime.datetime(2022, 5, 23, 20, 59, 26),
 datetime.datetime(2022, 5, 23, 20, 59, 55),
 datetime.datetime(2022, 5, 23, 22, 17, 18),
 datetime.datetime(2022, 5, 23, 22, 25, 51),
 datetime.datetime(2022, 5, 23, 22, 26, 20),
 datetime.datetime(2022, 5, 23, 22, 26, 49),
 datetime.datetime(2022, 5, 23, 22, 27, 18),
 datetime.datetime(2022, 5, 23, 22, 27, 47),
 datetime.datetime(2022, 5, 23, 22, 28, 16),
 datetime.datetime(2022, 5, 23, 22, 28, 45),
 datetime.datetime(2022, 5, 23, 22, 29, 14),
 datetime.datetime(2022, 5, 23, 22, 29, 43),
 datetime.datetime(2022, 5, 23, 22, 30, 13),
 datetime.datetime(2022, 5, 23, 22, 30, 42),
 datetime.datetime(2022, 5, 23, 22, 31, 11),
 datetime.datetime(2022, 5, 23, 22, 31, 40),
 datetime.d

In [10]:
# get the time of the file by grabbing the corresponding numbers from the file name
file[-15:-3]
file_dt = dt.strptime(file[-15:-3], '%y%m%d%H%M%S') # converts numbers to datetime
file_dt

datetime.datetime(2022, 5, 24, 0, 3, 9)

In [110]:
vad_ka1_times_new = []
for new_time_grab in range(len(vad_fileska1)):
    new_file = vad_fileska1[new_time_grab]
    new_time_yoink = new_file[49:-15]
    #time_yoink_datetime = dt.strptime(time_yoink, '%y%m%d%H%M%S')
    vad_ka1_times_new.append(new_time_yoink)

In [111]:
vad_ka1_times_new

['2018',
 '2018',
 '2019',
 '2019',
 '2020',
 '2056',
 '2057',
 '2057',
 '2057',
 '2058',
 '2058',
 '2059',
 '2059',
 '2130',
 '2131',
 '2131',
 '2132',
 '2132',
 '2133',
 '2135',
 '2135',
 '2136',
 '2136',
 '2137',
 '2137',
 '2217',
 '2217',
 '2218',
 '2218',
 '2219',
 '2219',
 '2220',
 '2220',
 '2221',
 '2221',
 '2222',
 '2222',
 '2223',
 '2223',
 '2224',
 '2224',
 '2225',
 '2225',
 '2226',
 '2226',
 '2227',
 '2227',
 '2228',
 '2228',
 '2229',
 '2229',
 '2230',
 '2230',
 '2231',
 '2231',
 '2232',
 '2232',
 '2233',
 '2233',
 '2234',
 '2234',
 '2235',
 '2235',
 '2236',
 '2236',
 '2237',
 '2237',
 '2238',
 '2238',
 '2239',
 '2239',
 '2240',
 '2240',
 '2241',
 '2241',
 '2242',
 '2243',
 '2244',
 '2245',
 '2245',
 '2246',
 '2247',
 '2248',
 '2249',
 '2250',
 '2250',
 '2251',
 '2255',
 '2255',
 '2256',
 '2256',
 '2257',
 '2257',
 '2258',
 '2258',
 '2259',
 '2259',
 '2300',
 '2300',
 '2301',
 '2301',
 '2302',
 '2302',
 '2303',
 '2304',
 '2305',
 '2306',
 '2306',
 '2307',
 '2308',
 '2309',
 

In [11]:
###### data for ka 2

ka2gps = pd.read_csv('/Users/juliabman/Desktop/research2024/GPS_Ka2_20220523.txt')
# scan data for first ka
ka2scan = pd.read_csv('/Users/juliabman/Desktop/research2024/20220523_Ka2_scan_log.txt', dtype=str)                    

# dealiased vad data created from VADS.ipynb for ka 2, but not bearing corrected
vad_fileska2 = sorted(glob.glob('/Users/juliabman/Desktop/research2024/product_raw_ka2/*.nc'))

# sanity checks to make sure they're loading in:
#print(vad_fileska2)
#print(ka2gps)

In [12]:
ka2gps['hhmmss[UTC]']

0        200239
1        200240
2        200241
3        200242
4        200243
          ...  
22101     25355
22102     25356
22103     25357
22104     25358
22105     25359
Name: hhmmss[UTC], Length: 22106, dtype: int64

In [13]:
tobac_file = '/Users/juliabman/Desktop/research2024/tobac_Save/Track.nc'
tobac_features_xr = xr.open_dataset(tobac_file)

In [14]:
tobac_features_xr

<xarray.Dataset>
Dimensions:                  (index: 851)
Coordinates:
  * index                    (index) int64 0 1 2 3 4 5 ... 846 847 848 849 850
Data variables: (12/15)
    frame                    (index) int64 ...
    idx                      (index) int64 ...
    hdim_1                   (index) float64 ...
    hdim_2                   (index) float64 ...
    num                      (index) int64 ...
    threshold_value          (index) int64 ...
    ...                       ...
    projection_y_coordinate  (index) float64 ...
    projection_x_coordinate  (index) float64 ...
    latitude                 (index) float64 ...
    longitude                (index) float64 ...
    cell                     (index) int64 ...
    time_cell                (index) timedelta64[ns] ...

In [15]:
# want to look at how frequently tobac times are taken
tobac_time_sample = tobac_features_xr['timestr'].values[40:60]

In [16]:
tobac_time_sample

array(['2022-05-23 20:38:29', '2022-05-23 20:38:29',
       '2022-05-23 20:38:29', '2022-05-23 20:42:59',
       '2022-05-23 20:42:59', '2022-05-23 20:42:59',
       '2022-05-23 20:47:31', '2022-05-23 20:47:31',
       '2022-05-23 20:47:31', '2022-05-23 20:47:31',
       '2022-05-23 20:52:02', '2022-05-23 20:52:02',
       '2022-05-23 20:52:02', '2022-05-23 20:56:32',
       '2022-05-23 20:56:32', '2022-05-23 20:56:32',
       '2022-05-23 20:56:32', '2022-05-23 21:01:16',
       '2022-05-23 21:01:16', '2022-05-23 21:01:16'], dtype='<U19')

In [17]:
# new = first_tobac_time[-8:]

In [18]:
# split = new.split(':')

In [19]:
# ''.join(split)

In [20]:
all_tobac_times = tobac_features_xr['timestr']

In [21]:
all_tobac_times

<xarray.DataArray 'timestr' (index: 851)>
array(['2022-05-23 20:04:20', '2022-05-23 20:04:20', '2022-05-23 20:04:20',
       ..., '2022-05-24 03:59:58', '2022-05-24 03:59:58',
       '2022-05-24 03:59:58'], dtype='<U19')
Coordinates:
  * index    (index) int64 0 1 2 3 4 5 6 7 8 ... 843 844 845 846 847 848 849 850

In [54]:
#all_tobac_timestr_no_dupes = [i for n, i in enumerate(all_tobac_times) if i not in all_tobac_times[:n]]

In [22]:
# np.abs((all_tobac_times - vad_ka1_times).total_seconds()) <= 300

Create a loop that will take the HH : MM : SS of each tobac time and put them in HHMMSS format.

In [23]:
tobac_times_list = []
for time in range(len(all_tobac_times)):
    file = all_tobac_times.values[time]
    time_select = file[-8:-3] # select the elements that represent the time
    time_split = time_select.split(':') # remove the : from the selected times
    time_join = ''.join(time_split) # join the time indeces with nothing separating them (aka, with '' joining them)
    tobac_times_list.append(time_join)

In [45]:
tobac_times_list[:5] # want only the hours and minutes, seconds is too precise 

['2004', '2004', '2004', '2004', '2004']

In [26]:
np.size(tobac_times_list)

851

Let's get rid of duplicates in the list with the enumerate() function:

In [47]:
tobac_times_list_no_dupes = [i for n, i in enumerate(tobac_times_list) if i not in tobac_times_list[:n]]

In [52]:
tobac_times_list_no_dupes

['2004',
 '2010',
 '2016',
 '2021',
 '2025',
 '2029',
 '2033',
 '2038',
 '2042',
 '2047',
 '2052',
 '2056',
 '2101',
 '2105',
 '2110',
 '2114',
 '2119',
 '2123',
 '2128',
 '2132',
 '2136',
 '2141',
 '2145',
 '2149',
 '2154',
 '2158',
 '2203',
 '2207',
 '2212',
 '2216',
 '2221',
 '2225',
 '2230',
 '2235',
 '2240',
 '2245',
 '2250',
 '2255',
 '2300',
 '2305',
 '2310',
 '2314',
 '2319',
 '2323',
 '2328',
 '2333',
 '2338',
 '2343',
 '2348',
 '2353',
 '2358',
 '0003',
 '0008',
 '0013',
 '0018',
 '0023',
 '0029',
 '0036',
 '0042',
 '0049',
 '0055',
 '0102',
 '0109',
 '0115',
 '0122',
 '0128',
 '0135',
 '0141',
 '0148',
 '0155',
 '0201',
 '0208',
 '0214',
 '0221',
 '0227',
 '0234',
 '0241',
 '0247',
 '0254',
 '0300',
 '0307',
 '0313',
 '0320',
 '0327',
 '0333',
 '0340',
 '0346',
 '0353',
 '0359']

In [49]:
np.size(tobac_times_list_no_dupes)

89

In [27]:
first_vad = xr.open_dataset(vad_fileska1[0])

In [28]:
first_vad

<xarray.Dataset>
Dimensions:                    (time: 2038, range: 555, sweep: 2)
Coordinates:
  * time                       (time) datetime64[ns] 2022-05-23T20:56:34 ... ...
  * range                      (range) float32 0.0 18.0 ... 9.954e+03 9.972e+03
    azimuth                    (time) float32 ...
    elevation                  (time) float32 ...
Dimensions without coordinates: sweep
Data variables: (12/27)
    total_power                (time, range) float32 ...
    reflectivity               (time, range) float32 ...
    velocity                   (time, range) float32 ...
    spectrum_width             (time, range) float32 ...
    corrected_velocity         (time, range) float32 ...
    normalized_coherent_power  (time, range) float32 ...
    ...                         ...
    longitude                  float64 ...
    altitude                   float64 ...
    time_coverage_start        |S32 ...
    time_coverage_end          |S32 ...
    time_reference             |S32 ...
    volume_number              int32 ...
Attributes: (12/15)
    Conventions:             CF/Radial instrument_parameters
    version:                 1.3
    title:                   
    institution:             
    references:              
    source:                  
    ...                      ...
    sigmet_task_name:        AF_VAD      
    sigmet_extended_header:  false
    time_ordered:            none
    rays_missing:            8
    history:                 
    field_names:             total_power, reflectivity, velocity, spectrum_wi...

In [29]:
first_vad['time']

<xarray.DataArray 'time' (time: 2038)>
array(['2022-05-23T20:56:34.000000000', '2022-05-23T20:56:34.000000000',
       '2022-05-23T20:56:34.000000000', ..., '2022-05-23T20:56:58.000000000',
       '2022-05-23T20:56:58.000000000', '2022-05-23T20:56:58.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time       (time) datetime64[ns] 2022-05-23T20:56:34 ... 2022-05-23T20:56:58
    azimuth    (time) float32 ...
    elevation  (time) float32 ...
Attributes:
    long_name:      time_in_seconds_since_volume_start
    standard_name:  time
    comment:        Coordinate variable for time. Time at the center of each ...

In [50]:
times_in_both_files = []
times_in_both_files_vads = []
for the_file in vad_fileska1:
    if (the_file[-9:-5] in tobac_times_list_no_dupes) != True:   #makes sure the times are the same from both files
        continue
        
    else:
        file_value = the_file[-9:-5]
        print(file_value)
        times_in_both_files.append(file_value)
        times_in_both_files_vads.append(the_file)

2056
2225
2230
2230
2240
2328
2328
0003


In [122]:
times_in_both_files

['/Users/juliabman/Desktop/research2024/product_raw_ka1/Ka1220523205632.nc',
 '/Users/juliabman/Desktop/research2024/product_raw_ka1/Ka1220523222551.nc',
 '/Users/juliabman/Desktop/research2024/product_raw_ka1/Ka1220523223013.nc',
 '/Users/juliabman/Desktop/research2024/product_raw_ka1/Ka1220523223042.nc',
 '/Users/juliabman/Desktop/research2024/product_raw_ka1/Ka1220523224032.nc',
 '/Users/juliabman/Desktop/research2024/product_raw_ka1/Ka1220523232822.nc',
 '/Users/juliabman/Desktop/research2024/product_raw_ka1/Ka1220523232852.nc',
 '/Users/juliabman/Desktop/research2024/product_raw_ka1/Ka1220524000309.nc']

In [123]:
np.size(times_in_both_files)

8

In [30]:
dealiased_data_ka1 = sorted(glob.glob('/Users/juliabman/Desktop/dealiased_data/Ka1*.nc'))

In [43]:
np.size(dealiased_data_ka1)

307

In [31]:
first = dealiased_data_ka1[0]

In [39]:
first[-17:-13]

'1819'

In [75]:
tobac_times_list_no_dupes

AttributeError: 'list' object has no attribute 'data'

In [70]:
times_in_both_files = []
times_in_both_files_vads = []

for the_file in dealiased_data_ka1:
    if (the_file[-17:-13] in tobac_times_list_no_dupes) != True:   #makes sure the times are the same from both files
        continue
        
    else:
        file_value = the_file[-17:-13]
        print(file_value)
        times_in_both_files.append(file_value)
        times_in_both_files_vads.append(the_file)

2119
2149
2250
0141
2025
2255
2021


In [71]:
np.size(times_in_both_files)

7

In [62]:
times_in_both_files_vads

['/Users/juliabman/Desktop/dealiased_data/Ka1220523222119_dealiased.nc',
 '/Users/juliabman/Desktop/dealiased_data/Ka1220523222149_dealiased.nc',
 '/Users/juliabman/Desktop/dealiased_data/Ka1220523222250_dealiased.nc',
 '/Users/juliabman/Desktop/dealiased_data/Ka1220523230141_dealiased.nc',
 '/Users/juliabman/Desktop/dealiased_data/Ka1220523232025_dealiased.nc',
 '/Users/juliabman/Desktop/dealiased_data/Ka1220523232255_dealiased.nc',
 '/Users/juliabman/Desktop/dealiased_data/Ka1220524002021_dealiased.nc']

Trying to do it the other way:
Get the tobac time list, which is shorter than the dealiased_data_ka1, and try to get times that match.

In [56]:
times_in_both_files = []
times_in_both_files_vads = []

for the_time in tobac_times_list:
    if (the_time in dealiased_data_ka1[-17:-13]) != True:   #makes sure the times are the same from both files
        continue
        
    else:
        file_value = dealiased_data_ka1[-17:-13]
        print(file_value)
        times_in_both_files.append(file_value)
        times_in_both_files_vads.append(the_time)

In [57]:
times_in_both_files

[]

Trying to retrieve tobac times that match the times in both files so that i can get their lat and lon. Is it not working because the entire vad file name is in times_in_both_files?

In [124]:
times_in_both_files_tobac = []
for tobac_time in tobac_times_list:
    if (tobac_time in times_in_both_files) != True:   #makes sure the times are the same from both files
        continue
        
    else:
        file_value = tobac_time
        print(tobac_time)
        
        times_in_both_files_tobac.append(tobac_time)
#print(len(times_in_both_files))

In [126]:
times_in_both_files_tobac

[]

In [127]:
np.size(times_in_both_files_tobac)

0

Read the lats and lons from the times in both files.

In [60]:
latitude_times_in_both_files = []
longitude_times_in_both_files = []
for read_netcdf in times_in_both_files_vads:
    read = pyart.io.read_cfradial(read_netcdf)
    latitude = read.latitude['data']
    longitude = read.longitude['data']
    latitude_times_in_both_files.append(latitude)
    longitude_times_in_both_files.append(longitude)

In [61]:
latitude_times_in_both_files

[masked_array(data=[43.53127012],
              mask=False,
        fill_value=1e+20),
 masked_array(data=[43.53127012],
              mask=False,
        fill_value=1e+20),
 masked_array(data=[43.53127012],
              mask=False,
        fill_value=1e+20),
 masked_array(data=[43.53127012],
              mask=False,
        fill_value=1e+20),
 masked_array(data=[43.53127012],
              mask=False,
        fill_value=1e+20),
 masked_array(data=[43.53127012],
              mask=False,
        fill_value=1e+20),
 masked_array(data=[43.53127012],
              mask=False,
        fill_value=1e+20)]

In [63]:
longitude_times_in_both_files

[masked_array(data=[-94.63158991],
              mask=False,
        fill_value=1e+20),
 masked_array(data=[-94.63158991],
              mask=False,
        fill_value=1e+20),
 masked_array(data=[-94.63158991],
              mask=False,
        fill_value=1e+20),
 masked_array(data=[-94.63158991],
              mask=False,
        fill_value=1e+20),
 masked_array(data=[-94.63158991],
              mask=False,
        fill_value=1e+20),
 masked_array(data=[-94.63158991],
              mask=False,
        fill_value=1e+20),
 masked_array(data=[-94.63158991],
              mask=False,
        fill_value=1e+20)]

In [64]:
tobac_times_list_no_dupes

['2004',
 '2010',
 '2016',
 '2021',
 '2025',
 '2029',
 '2033',
 '2038',
 '2042',
 '2047',
 '2052',
 '2056',
 '2101',
 '2105',
 '2110',
 '2114',
 '2119',
 '2123',
 '2128',
 '2132',
 '2136',
 '2141',
 '2145',
 '2149',
 '2154',
 '2158',
 '2203',
 '2207',
 '2212',
 '2216',
 '2221',
 '2225',
 '2230',
 '2235',
 '2240',
 '2245',
 '2250',
 '2255',
 '2300',
 '2305',
 '2310',
 '2314',
 '2319',
 '2323',
 '2328',
 '2333',
 '2338',
 '2343',
 '2348',
 '2353',
 '2358',
 '0003',
 '0008',
 '0013',
 '0018',
 '0023',
 '0029',
 '0036',
 '0042',
 '0049',
 '0055',
 '0102',
 '0109',
 '0115',
 '0122',
 '0128',
 '0135',
 '0141',
 '0148',
 '0155',
 '0201',
 '0208',
 '0214',
 '0221',
 '0227',
 '0234',
 '0241',
 '0247',
 '0254',
 '0300',
 '0307',
 '0313',
 '0320',
 '0327',
 '0333',
 '0340',
 '0346',
 '0353',
 '0359']

In [66]:
dealiased_ka1_times = []
for time_grab in range(len(dealiased_data_ka1)):
    file = dealiased_data_ka1[time_grab]
    time_yoink = file[43:-13]
    #time_yoink_datetime = datetime.strptime(time_yoink, '%y%m%d%H%M%S')
    dealiased_ka1_times.append(time_yoink)

In [69]:
dealiased_ka1_times

['220523201819',
 '220523201849',
 '220523201919',
 '220523201948',
 '220523202019',
 '220523205632',
 '220523205701',
 '220523205730',
 '220523205759',
 '220523205828',
 '220523205857',
 '220523205926',
 '220523205955',
 '220523213052',
 '220523213123',
 '220523213152',
 '220523213222',
 '220523213253',
 '220523213323',
 '220523213503',
 '220523213532',
 '220523213603',
 '220523213649',
 '220523213716',
 '220523213743',
 '220523221718',
 '220523221747',
 '220523221817',
 '220523221847',
 '220523221918',
 '220523221948',
 '220523222018',
 '220523222048',
 '220523222119',
 '220523222149',
 '220523222219',
 '220523222250',
 '220523222321',
 '220523222351',
 '220523222420',
 '220523222452',
 '220523222522',
 '220523222551',
 '220523222620',
 '220523222649',
 '220523222718',
 '220523222747',
 '220523222816',
 '220523222845',
 '220523222914',
 '220523222943',
 '220523223013',
 '220523223042',
 '220523223111',
 '220523223140',
 '220523223209',
 '220523223238',
 '220523223307',
 '220523223336

In [68]:
tobac_features_xr['latitude'].data

array([32.36817342, 32.6809758 , 32.7166771 , 32.76517226, 32.78410632,
       32.80010489, 32.84432997, 32.38490837, 32.56757967, 32.76015088,
       32.78597801, 32.85314655, 32.88587211, 32.40713171, 32.71452275,
       32.80657897, 32.87523682, 32.90594997, 32.79207596, 32.29996895,
       32.43081958, 32.78486818, 32.77911726, 32.91826327, 32.90312963,
       32.30529179, 32.39664528, 32.47724774, 32.80449884, 32.92617352,
       32.92132748, 32.31185939, 32.49651682, 32.82001167, 32.92939234,
       32.94496521, 32.31282865, 32.84153001, 32.93791337, 32.97638755,
       32.84873933, 32.95078067, 33.00239168, 32.60562931, 32.95828745,
       33.03310555, 32.6218254 , 32.93904032, 32.96386085, 33.06245177,
       32.63920491, 32.97457525, 33.07545755, 32.65670849, 32.8952915 ,
       32.98310243, 33.08541553, 32.90810633, 32.98790555, 33.11435665,
       33.13893221, 33.00198198, 32.29783729, 32.99896847, 32.95292014,
       33.1599002 , 32.30060616, 33.17182326, 33.00721062, 33.18